In [ ]:
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import scipy as sp
import pandas as pd
from netCDF4 import Dataset
import dask
import subprocess
import platform

# Variable Glossary
In the following cell you can set the values of the variables relevant to the postprocess. The details of each variable are included below. Note that any variable included in the model should be given the same value in the postprocess. For example, if the model used zw = 42 and kmax = 11, you should use zw = 42 and kmax = 11 below.


## Standard Variables
In most cases it is only necessary to set values for the standard variables.

**zw** is the zonal wave number. For standard use, zw should be set to the value of either 42, 63, or 124. Setting the value for zw also sets default values jmax and imax.
<br>Set zw = 42, to set jmax = 64 and imax = 128.
<br>Set zw = 63, to set jmax = 96 and imax = 192.
<br>Set zw = 124, to set jmax = 188 and imax = 376.
<br>Each of these variables (jmax and imax) that is given a value in the advanced variables section will instead use that value.

**kmax** is the number of vertical levels. The value of kmax should be 11 or 26 for standard use.

**Expname** is the name you gave to your experiment. When the data was saved to your computer, it was saved in a folder with this name. This variable should be set to the same experiment name you used when running the model.

**DataSetname** is the name of the data set. This will be used to name files that are created by this program.

**Dataname** is the name of the data. This is what will be used to label the data in the charts generated by this program.

**dayst** is the number of days over which the data spans. This number will be used to label the data as it's output to files and will also be used to run this program and display the data.


## Advanced Variables
While most cases only require setting the standard variables, some cases might require setting some or all advanced variables as well. The following variables should only be changed from their default value if a specific behavior is desired. An advanced variable set to the value of None will use the default case.

**jmax** is the number of Gaussian latitudes. jmax = imax/2

**imax** is the number of longitude grid points. imax >= 3 * zw + 1. imax must be an even number.

**custom_path** is the full path of the folder in which you saved your data. If custom_path is set, expname is ignored. If you used a custom_path when running the model, you must use the same custom_path here to access the same data.

**custom_kmax** is used to safeguard against using unexpected values for the kmax. If custom_kmax is set, it will be used instead of kmax. By default the program only supports kmax with a value of either 11 or 26. Other values are implementable, but the user must modify subs1_utils.py routine bscst. If unclear email bkirtman@miami.edu for clarification.

In [ ]:
# Set postprocess parameters.

# Standard Variables
zw = 42
kmax = 11
Expname = 'Control'
DataSetname = 'vvel'
Dataname = 'v'
dayst = 90

# Advanced Variables
imax = None
jmax = None
custom_path = None
custom_kmax = None

In [ ]:
# Set Dependent Variables

# Set value of kmax if custom_kmax is used.
if not(custom_kmax is None):
    kmax = custom_kmax
    print("Using custom value for kmax:", kmax)
# Otherwise check value for kmax.
elif kmax!=11 and kmax!=26:
    raise Exception("Unexpected value for kmax. Use custom_kmax and note that other values are implementable, but the user must modify subs1_utils.py routine bscst. If unclear email bkirtman@miami.edu for clarification.")

# Check value for zw.
# Afterwards, set jmax and imax values based on the value given to zw.
# If a value is already given for one of the listed variables, use that instead
match zw:
    case 42:
        jmax = 64 if (jmax is None) else jmax
        imax = 128 if (imax is None) else imax
    case 63:
        jmax = 96 if (jmax is None) else jmax
        imax = 192 if (imax is None) else imax
    case 124:
        jmax = 188 if (jmax is None) else jmax
        imax = 376 if (imax is None) else imax
    case _:
        if (jmax is None) or (imax is None):
            raise Exception("Unexpected value for zw. Other values are implementable, but the user must specify values for jmax and imax in the advanced variables section.")

print("zw =", zw,
      "\nkmax =", kmax,
      "\njmax =", jmax,
      "\nimax =", imax,
      "\ndayst =", dayst)

In [ ]:
# Set datapath.

# If custom_path was set, use that as the datapath.
# Otherwise create an appropriate datapath for the user's operating system.
user_platform = platform.system() if (custom_path is None) else "Custom Path"
print("Setting output datapath for", user_platform)
datapath = ''
match user_platform:
    case 'Custom Path':
        datapath = custom_path
    case 'Windows':
        foo = str(subprocess.check_output(['whoami']))
        end = len(foo) - 5
        uname = foo[2:end].split("\\\\")[1]
        datapath = "C:\\Users\\" + uname + "\\Documents\\AGCM_Experiments\\" + Expname + "\\"
    case 'Darwin':
        foo = str(subprocess.check_output(['whoami']))
        end = len(foo) - 3
        uname = foo[2:end]
        datapath = '/Users/' + uname + '/Work/AGCM/AGCM/tmp4/' + Expname + '/'
    case _:
        raise Exception("Use case for this system/OS is not implemented. Consider using custom_path in the advanced variables.")

# Set stamp for file names
stamp = 'days_1-' + str(dayst)

print("datapath =", datapath,
      "\nstamp =", stamp)

In [ ]:
fps = datapath+'lnps_1*.nc' # always need surface pressure
dps = xarray.open_mfdataset(fps,decode_times=True, parallel=True)
#
#
fdata = datapath+DataSetname+'_1*.nc'
ddata = xarray.open_mfdataset(fdata,decode_times=True, parallel=True)

In [ ]:
fps

In [ ]:
dps

In [ ]:
def sigma_to_press(din,ps,lev,kmax):
    din = np.flip(din,axis=0)
    dout = np.zeros((11,jmax,imax)) # '11' here is the manditory pressure levels
    lev = np.flip(lev,axis=0)
    surfp = (np.exp(ps))*1000.0
    plev = [1000.0,900.0,800.0,700.0,600.0,500.0,400.0,300.0,200.0,100.0,20.0]
    #
    # make plev, press, isiglev, press_up, press_down in to arrays so no loops over i & j
    plevR15 = np.zeros((11,jmax,imax))
    for k in range(11):
        plevR15[k,:,:] = plev[k]
    #
    #
    # Find the appropriate pressure level
    #
    found = np.zeros((11,jmax,imax))
    data_up = np.zeros((11,jmax,imax))
    data_dn = np.zeros((11,jmax,imax))
    press_up = np.zeros((11,jmax,imax))
    press_dn = np.zeros((11,jmax,imax))
    for kk in range(11): # "11" here is the number of output pressure levels
        for kkk in range(kmax-1):
            sigp_up = (lev[kkk+1].values)*(surfp.values)
            sigp_dn = (lev[kkk].values)*(surfp.values)
            isiglev_up = ( sigp_up < plevR15[kk] )
            isiglev_dn = ( sigp_dn > plevR15[kk] )
            isiglev_up = 1*isiglev_up
            isiglev_dn = 1*isiglev_dn
            check = ((isiglev_up + isiglev_dn) == 2)
            check = (1*check) + found[kk]
            check = ( check == 1)
            check = 1*check
            found[kk] = check*kkk + (1-check)*found[kk]
        #
        for kkk in range(kmax-1):
            sigp_up = (lev[kkk+1].values)*(surfp.values)
            sigp_dn = (lev[kkk].values)*(surfp.values)
            check = (found[kk] == kkk)
            check = 1*check
            data_up[kk] = check*din[kkk+1]+(1-check)*data_up[kk]
            data_dn[kk] = check*din[kkk]+(1-check)*data_dn[kk]
            press_up[kk] = check*sigp_up+(1-check)*press_up[kk]
            press_dn[kk] = check*sigp_dn+(1-check)*press_dn[kk]
        #
    #
    press_up = np.where(press_up <= 0.0, 0.1, press_up)
    press_dn = np.where(press_dn <= 0.0, 0.1, press_dn)
    tmp1 = np.zeros((11,jmax,imax))
    for kk in range(11):
        xup = np.log(plevR15[kk]) - np.log(press_dn[kk])
        xdn = np.log(press_up[kk]) - np.log(press_dn[kk])
        yup = np.log(press_up[kk]) - np.log(plevR15[kk])
        tmp = ((xup/xdn)*data_up[kk])+((yup/xdn)*data_dn[kk])
        check = (found[kk] > 0)
        check = 1*check
        tmp1[kk] = check
        dout[kk] = check*tmp + (1-check)*dout[kk]
#        dout[kk] = np.where(check <= 0.0, None, dout[kk]) # This sets below surface to
                                                          # undefined
    #
    for k in range(9): ### if below ground use lowest sigma level
        dout[k] = tmp1[k]*dout[k] + (1-tmp1[k])*din[0].values
    #
    return dout

In [ ]:
#
# Create Data Array for Control Pressure level Data geopotenial, temp, u & v
# 
#
#
#
tmp = (dayst,11,jmax,imax) #### "11" here corresponds to standard pressure levels not to model levels
dout = np.zeros(tmp)
siglevs = ddata['lev']
for k in range (dayst):
    vv = ddata[Dataname][k,:,:,:]
    ps = dps.lnps[k,:,:]
    vv = vv.compute()
    ps = ps.compute()
    dout[k] = sigma_to_press(vv,ps,siglevs,kmax)
lats = ddata['lat'].values
lons = ddata['lon'].values
plev = [1000.0,900.0,800.0,700.0,600.0,500.0,400.0,300.0,200.0,100.0,20.0]
times = ddata['time']
dData = xarray.Dataset({Dataname: (['time','lev','lat','lon'],dout)},
                        coords={'time': times,'lev':plev, 'lat': lats, 'lon': lons})

In [ ]:
dData

In [ ]:
dData.to_netcdf(datapath+DataSetname+'_Pressure_'+stamp+'.nc')

In [ ]:
dData[Dataname][0,0,:,:].plot()

In [ ]:
dData[Dataname][29,3,:,:].plot()

In [ ]:
dData[Dataname][0,:,45,64].plot()